In [ ]:
# Welcome

import os
import numpy as np
import pandas as pd
import scipy
from mip import *

In [ ]:
# Read participants to be matched

tobematched_df = pd.read_csv('February Participants.csv', sep=',')
# tobematched_df = pd.read_csv('Test Participants.csv', sep=',')

tobematched_df.head()

In [ ]:
# Read past matches

previousmatches_df = []

for file in os.listdir('Past Matches'):
    previousmatches_df.append(pd.read_csv(os.path.join(os.getcwd(),'Past Matches',file)))
    
# for file in os.listdir('Test Matches'):
#     previousmatches_df.append(pd.read_csv(os.path.join(os.getcwd(),'Test Matches',file)))
    
previousmatches_df = pd.concat(previousmatches_df)

print(previousmatches_df.shape)
previousmatches_df.head()

In [ ]:
# Remove people who are not currently participating

# remove the rows of the participant that opted out
previousmatches_df = previousmatches_df[previousmatches_df['Name'].isin(tobematched_df['Name'])]

# remove the mentions of the participant that opted out from others' matches
previousmatches_df.loc[~previousmatches_df['Match 1'].isin(tobematched_df['Name']),'Match 1'] = np.nan
previousmatches_df.loc[~previousmatches_df['Match 2'].isin(tobematched_df['Name']),'Match 2'] = np.nan
previousmatches_df.loc[~previousmatches_df['Match 3'].isin(tobematched_df['Name']),'Match 3'] = np.nan

print(previousmatches_df.shape)
previousmatches_df.head()

In [ ]:
# Get match eligibility (have not been matched together before)

# Make a dictionary for the participants (to map participant names to indices)
participants_indices = dict()

# Filling in the participants indices
for i,row in tobematched_df.iterrows():
    participants_indices[row['Name']] = i

# Make a dictionary for the previous matches (based on the indices made previously)
previous_matches = dict()
for i,row in tobematched_df.iterrows():
    previous_matches[i]=set()

# Assign the previous matches to the particpants
# NOTE: we need to add Match 3 for the real deal
for i,row in previousmatches_df.iterrows():
    name_index = participants_indices[row['Name']]
    if name_index not in previous_matches:
        previous_matches[name_index] = set()
    if not pd.isnull(row['Match 1']):
        previous_matches[name_index].add(participants_indices[row['Match 1']])
    if not pd.isnull(row['Match 2']):
        previous_matches[name_index].add(participants_indices[row['Match 2']])
    if not pd.isnull(row['Match 3']):
        previous_matches[name_index].add(participants_indices[row['Match 3']])
        
# Show the results (the people they CANNOT be matched with since they have already been matched with them)
print(participants_indices)
print(previous_matches)

In [ ]:
# Get match eligibility (speak the same language)

# Use the participants_indices from the previous cell

# Make a dictionary for people's languages (based on the indices made previously)
languages_matches = dict()

#Assign people who speak the same language
for i,participant1 in tobematched_df['Languages'].items():
    # Get languages that the people speak (set = group of unique values, unordered)
    participant1_languages = set(participant1.split(';'))
    # print(participant1_languages)
    languages_matches[i] = set()
    for j,participant2 in tobematched_df['Languages'].items():
        if i==j:
            continue
        # Same as for participant1_languages
        participant2_languages = set(participant2.split(';'))
        # Look for the intersectios (things in common) between both participants' list of languages
        common_languages = participant1_languages.intersection(participant2_languages)
        # Check if there is a language in common
        if common_languages:
            languages_matches[i].add(j)
            
# Show the results (the people that can be matched together because they can actually communicate)
print(languages_matches)

In [ ]:
# Get match eligibility (years of experience/preferences)

# Use the same participants_indices from previous sections

# Make a dictionary for people's experience (0=student; 1=0-5 years; 2=6-10 years;3=11+years)
experience = {'Student':0,'0-5 years':1,'6-10 years':2,'11+ years':3}
# Switch the things in the Experience column to be what's in the dictionary
# lambda is a function that you can define on the fly in one line
tobematched_df['Years of Experience'] = tobematched_df['Years of Experience'].map(lambda x:experience[x])

# Make a dictionary to match according to the elegible matches
experience_matches = dict()

#Assign the numbers to the people
for i,participant1 in tobematched_df.iterrows():
    participant1_pref = set(participant1['Match Preference'].split(';'))
    participant1_exp = participant1['Years of Experience']
    experience_matches[i] = set()
    for j,participant2_exp in tobematched_df['Years of Experience'].items():
        if i==j:
            continue
        isValid = False
        isValid = True if 'less' in participant1_pref and participant2_exp < participant1_exp else isValid
        isValid = True if 'same' in participant1_pref and participant2_exp == participant1_exp else isValid
        isValid = True if 'more' in participant1_pref and participant2_exp > participant1_exp else isValid
        
        if isValid:
            experience_matches[i].add(j)
        
print(experience_matches)

In [ ]:
# Make objective function coefficients

# Number of i times number of j
numberMatchVar = tobematched_df.shape[0] * tobematched_df.shape[0]

# Number of i
numberDevVar = tobematched_df.shape[0]

c = -np.ones(numberMatchVar + numberDevVar)

c[numberMatchVar:]*=-1.5

print(numberMatchVar, numberDevVar)
print(c.shape)
print(c)

In [ ]:
# Make equality constraint coefficients
# Reciprocity & with themselves

A_equality = np.zeros((numberMatchVar, numberMatchVar + numberDevVar))

for i in range(numberMatchVar):
    j = i // numberDevVar + numberDevVar * (i % numberDevVar)
    A_equality[i,j] = -1
    A_equality[i,i] = 1

print(A_equality.shape)
print(A_equality)

In [ ]:
# Make equality constraint right-hand-side

b_equality = np.zeros(numberMatchVar)

print(b_equality.shape)
print(b_equality)

In [ ]:
# Make inequality constraint coefficients

# Maximum number of matches
A_inequality = np.zeros((2 * numberDevVar + 2 * numberMatchVar, numberMatchVar + numberDevVar))

for i in range(numberDevVar):
    A_inequality[i,i*numberDevVar:(i+1)*numberDevVar] = np.ones(numberDevVar)

    
# Matching with new people + languages
offset_row = numberDevVar
    
A_inequality[offset_row:offset_row+numberMatchVar,:numberMatchVar] = np.eye(numberMatchVar)


# Everybody needs a match
offset_row += numberMatchVar

for i in range(numberDevVar):
    A_inequality[offset_row+i,i*numberDevVar:(i+1)*numberDevVar] = -np.ones(numberDevVar)

    
# needs to be redone (eligible/preferred matches)
offset_row += numberDevVar

A_inequality[offset_row:offset_row + numberMatchVar,:numberMatchVar] = np.eye(numberMatchVar)

for i in range(numberDevVar):
    A_inequality[offset_row + i * numberDevVar:offset_row + (i + 1) * numberDevVar,numberMatchVar+i] = -np.ones(numberDevVar)

    
# Variables must be between 0 and 1
    
print(A_inequality.shape)
print(A_inequality[:numberDevVar,:])
print()
print(A_inequality[numberDevVar:numberDevVar + numberMatchVar,:])
print()
print(A_inequality[numberDevVar + numberMatchVar:2 * numberDevVar + numberMatchVar,:])
print()
print(A_inequality[2 * numberDevVar + numberMatchVar:,:])

In [ ]:
# Make the inequality constraint right-hand side


b_inequality = np.zeros(2 * numberDevVar + 2 * numberMatchVar)

# Right-hand side for the max number of matches
b_inequality[:numberDevVar] = tobematched_df['Number of matches']

# Right-hand side for the previous matches + languages
offset_row = numberDevVar
b_inequality[offset_row:offset_row+numberMatchVar] = np.fromfunction(np.vectorize(lambda i,j: (i not in previous_matches or 
                                                                                  j not in previous_matches[i])
                                                                                  and j in languages_matches[i]),
                                                                     (numberDevVar,numberDevVar), dtype=np.float).flatten()

# Right-hand side for everyone needs a match
offset_row += numberMatchVar
b_inequality[offset_row:offset_row+numberDevVar] = -1

# Right-hand side for preferred matches
offset_row += numberDevVar
b_inequality[offset_row:] = np.fromfunction(np.vectorize(lambda i,j : j in experience_matches[i]),
                                            (numberDevVar, numberDevVar), dtype=np.float).flatten()

print(b_inequality.shape)
print(b_inequality[:numberDevVar])
print()
print(b_inequality[numberDevVar:numberDevVar + numberMatchVar])
print()
print(b_inequality[numberDevVar + numberMatchVar:2 * numberDevVar + numberMatchVar])
print()
print(b_inequality[2 * numberDevVar + numberMatchVar:])

In [ ]:
m = Model()

x = [m.add_var(var_type=BINARY) for i in range(numberMatchVar)]
d = [m.add_var() for i in range(numberDevVar)]
v = x+d

for row in range(A_inequality.shape[0]):
    m += xsum(A_inequality[row,column]*v[column] for column in range(A_inequality.shape[1])) <= b_inequality[row]
    
for row in range(A_equality.shape[0]):
    m += xsum(A_equality[row,column]*v[column] for column in range(A_equality.shape[1])) == b_equality[row]

m.objective = xsum(c[i]*v[i] for i in range(c.shape[0]))

m.max_gap = 0.01
status = m.optimize(max_seconds=3600)
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print('solution:')
    for i in m.vars:
        if abs(i.x) > 1e-6: # only printing non-zeros
          print('{} : {}'.format(i.name, i.x))

In [ ]:
print(status)

In [ ]:
# Map the results back to names

# Make them get back into integers
solution = [int(i.x) for i in m.vars]
solution = np.array(solution, dtype=np.int)

# Make a dictionnary of the matches
final_matches = dict()

for i, participant1 in tobematched_df['Name'].items():
    final_matches[participant1] = set()
    for j,participant2 in tobematched_df['Name'].items():
        if solution[i*numberDevVar+j]:
            final_matches[participant1].add(participant2)

print(solution)
print(final_matches)

In [ ]:
# Make it mail-merge friendly

# Make the dataframe with the results
# Get the list of participants
matches_output = tobematched_df[['Name','Email']]

# Map  the years of experience back
experience2 = {0:'Student',1:'0-5 years',2:'6-10 years',3:'11+ years'}
# Switch the things in the Experience column to be what's in the dictionary
# lambda is a function that you can define on the fly in one line
tobematched_df['Years of Experience'] = tobematched_df['Years of Experience'].map(lambda x:experience2[x])


# Initialize the other columns
matches_output['Match1Name'] = ''
matches_output['Match1Email'] = ''
matches_output['Match1Years'] = ''
matches_output['Match2Name'] = ''
matches_output['Match2Email'] = ''
matches_output['Match2Years'] = ''
matches_output['Match3Name'] = ''
matches_output['Match3Email'] = ''
matches_output['Match3Years'] = ''

# Fill in the colomns by iterating over the matches
for i,row in matches_output.iterrows():
    matches = tobematched_df.iloc[np.argwhere(solution[i * numberDevVar:(i+1)*numberDevVar]).flatten()]
    matches.reset_index(inplace=True)
    print (matches)
    for j,row2 in matches.iterrows():
        print (j)
        row[f'Match{j+1}Name'] = row2['Name']
        row[f'Match{j+1}Email'] = row2['Email']
        row[f'Match{j+1}Years'] = row2['Years of Experience']

print(matches_output)

In [ ]:
# Print the solution to a file

matches_output.to_csv('February Matches.csv', sep=',')